In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("DateTransformation").getOrCreate()

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DecimalType, StringType, StructField, StructType
from 
import re

# Define the schema for the returned STRUCT type
measure_unit_schema = StructType([
    StructField("Measure", StringType(), True),
    StructField("UnitOfMeasure", StringType(), True),
    StructField("PackageUnits", StringType(), True)
])

# Register the UDFs
infer_and_transform_date_udf = udf(infer_and_transform_date, StringType())
transform_price_udf = udf(transform_price, DecimalType())
remove_special_characters_udf = udf(remove_special_characters, StringType())
separate_camel_case_udf = udf(separate_camel_case, StringType())
transform_provider_name_udf = udf(transform_provider_name, StringType())
extract_measure_and_unit_udf = udf(extract_measure_and_unit, measure_unit_schema)

In [0]:
# Read the table into a DataFrame
df = spark.read.table("workspace.default.products_from_csv")

In [0]:
from pyspark.sql.functions import col, initcap, when

# Convert data types using the UDFs
df = df.withColumn("RawPrice", col("Price")) \
       .withColumn("CleanPrice", transform_price_udf(col("Price"))) \
       .withColumn("RawLastReviewDt", col("LastReviewDt")) \
       .withColumn("CleanLastReviewDt", infer_and_transform_date_udf(col("LastReviewDt"))) \
       .withColumn("RawDescription", col("Description")) \
       .withColumn("CleanDescription", remove_special_characters_udf(col("Description"))) \
       .withColumn("CleanProviderName", initcap(transform_provider_name_udf(col("ProviderName")))) \
       .withColumn("IsValidPrice", when(col("Price").isNull(), False).otherwise(True))

df.display()

In [0]:
from pyspark.sql.functions import lower

# Convert data types using the UDFs
df = df.withColumn("MeasureUnit", extract_measure_and_unit_udf(col("Description"))) \
       .withColumn("Measure", col("MeasureUnit.Measure")) \
       .withColumn("UnitOfMeasure", col("MeasureUnit.UnitOfMeasure")) \
       .withColumn("PackageUnits", col("MeasureUnit.PackageUnits")) \
       .withColumn("UnitOfMeasure", lower(col("UnitOfMeasure")))

df.display()

In [0]:
prompt = """Contexto: 
    * Eres un API que recibirá descripciones breves de productos, principalmente en español. Estos productos provienen de catálogos de diferentes provedores y sus descripciones no son consistentes en cuanto al uso de preposiciones, abreviaciones, unidades de medida, cantidad por unidad y unidades por paquete. Por tanto, trata de extraer esta información, además del nombre correcto del producto para evitar dichas ambiguedades del provedor. 

    Reglas:
    * Si el nombre del producto contiene palabras en ingles y estas tienen algún error ortográfico, corrigelo pero nunca traduzcas. 
    * Nunca omitas palabras presentes en el nombre del producto. 
    * Si no tienes certeza, un humano debe revisar el nombre. 
    * Si la descripción del producto contiene la unidad de medida, cambila por su abbreviación.
    * Debe inferir el nombre del producto, la unidad de medida, la medida según la unidad de medida y la cantidad de unidades por paquete (algunos provedores venden por paquetes y no por unidades).
    * Si los valores de las propiedades numéricas no son claros, use 1 por defecto. 
    * Si el producto no especifica la unidad de medida, intente inferirla segun el tipo de producto. Por ejemplo, si el producto es una 'Coca Cola pequeña', la unidad de medida sería 'ml' de mililitros, y si la Coca Cola es grande, la unidad de medida sería 'l' de litros. 
    * Si no se puede saber cual es la unidad de medida, utilize 'u' que significa 'unidad' y funciona como fallback. 
    * Si un humando debe revisar el nombre del producto, 'needsHumanVerification' debe ser true. 
    * 'B' significa 'Barato', asi que cambialo si aparece en la descripción del producto.

    Formato de la respuesta:
    * Su respuesta será un objeto JSON con las propiedades 'productName', 'unitType', 'units', 'packageUnits' y 'needsHumanVerification'. 
    * Si se tiene certeza, y no necesitas que un humano revise, el objeto JSON no debe contener la propieded 'needsHumanVerification'.
    * El 'productName' es del tipo string.
    * El 'unitType' es del tipo string y debe abreviarse en español y en minúsculas. 
    * El 'units' es un número decimal
    * El 'packageUnits' siempre es un número entero
    * RESPONDE UNICAMENTE EL JSON OBJECT, NO EXPLICACION, NO PREFIJOS, NADA MAS
    
    Esta es la descripción del producto: 
    """

In [0]:
df.createOrReplaceTempView("products_cleaned")

In [0]:
# Escape single quotes and replace newlines in the prompt
prompt_sql = prompt.replace("'", "''").replace("\n", " ")

df_out = df.selectExpr(f"ai_query('databricks-meta-llama-3-3-70b-instruct', CONCAT('{prompt_sql}', Description), modelParameters => named_struct('max_tokens', 100, 'temperature', 0.5)) as summary"
).limit(10)

display(df_out)

In [0]:
df.display()